In [7]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'covid_county_formatted'
cursor = db.sustaindb[collection].find()
df = pd.DataFrame(list(cursor))
df

,_id,date,county,state,fips,cumulative_cases,cumulative_deaths,cases,deaths,GISJOIN,formatted_date
0,605420df8dd202bf891eb048,2020-03-27,Autauga,Alabama,01001,6,0.0,0,0,G0100010,2020-03-27
1,605420df8dd202bf891eb049,2020-03-25,Autauga,Alabama,01001,4,0.0,3,0,G0100010,2020-03-25
2,605420df8dd202bf891eb04a,2020-03-28,Autauga,Alabama,01001,6,0.0,0,0,G0100010,2020-03-28
3,605420df8dd202bf891eb04b,2020-03-29,Autauga,Alabama,01001,6,0.0,0,0,G0100010,2020-03-29
4,605420df8dd202bf891eb04c,2020-03-30,Autauga,Alabama,01001,7,0.0,1,0,G0100010,2020-03-30
...,...,...,...,...,...,...,...,...,...,...,...
1064945,605420f18dd202bf892ef039,2021-02-24,St. Thomas,Virgin Islands,78030,1324,15.0,10,0,NaN,2021-02-24
1064946,605420f18dd202bf892ef03a,2021-03-04,St. Thomas,Virgin Islands,78030,1354,15.0,8,0,NaN,2021-03-04
1064947,605420f18dd202bf892ef03b,2021-03-07,St. Thomas,Virgin Islands,78030,1358,15.0,0,0,NaN,2021-03-07
1064948,605420f18dd202bf892ef03c,2021-03-05,St. Thomas,Virgin Islands,78030,1358,15.0,4,0,NaN,2021-03-05


In [12]:
import time

df_cases = df[["date", "GISJOIN", "cases"]].rename(columns = {
    'date': 'ds',
    'cases': 'y'
})

all_gis_joins = df_cases['GISJOIN'].unique()
print(f'#GISJOINs: {len(all_gis_joins)}')

all_dfs = []

count = 0
time1 = time.monotonic()
for gis_join in all_gis_joins:
    count += 1
    if count % 20 == 0:
        print(count, end=', ')
    all_dfs.append(df_cases.loc[df_cases['GISJOIN'] == gis_join])
    
print(f'Time to separate pandas dataframes by GISJOIN value: {time.monotonic() - time1}')
print(f'#Dataframes: {len(all_dfs)}')

#GISJOINs: 3116
20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 520, 540, 560, 580, 600, 620, 640, 660, 680, 700, 720, 740, 760, 780, 800, 820, 840, 860, 880, 900, 920, 940, 960, 980, 1000, 1020, 1040, 1060, 1080, 1100, 1120, 1140, 1160, 1180, 1200, 1220, 1240, 1260, 1280, 1300, 1320, 1340, 1360, 1380, 1400, 1420, 1440, 1460, 1480, 1500, 1520, 1540, 1560, 1580, 1600, 1620, 1640, 1660, 1680, 1700, 1720, 1740, 1760, 1780, 1800, 1820, 1840, 1860, 1880, 1900, 1920, 1940, 1960, 1980, 2000, 2020, 2040, 2060, 2080, 2100, 2120, 2140, 2160, 2180, 2200, 2220, 2240, 2260, 2280, 2300, 2320, 2340, 2360, 2380, 2400, 2420, 2440, 2460, 2480, 2500, 2520, 2540, 2560, 2580, 2600, 2620, 2640, 2660, 2680, 2700, 2720, 2740, 2760, 2780, 2800, 2820, 2840, 2860, 2880, 2900, 2920, 2940, 2960, 2980, 3000, 3020, 3040, 3060, 3080, 3100, Time to separate pandas dataframes by GISJOIN value: 229.87625111918896
#Dataframes: 3116


In [15]:
import pickle

f = open('pickles/all_dfs.pkl', 'wb')
time1 = time.monotonic()
pickle.dump(all_dfs, f)

print(f'Time to dump all_dfs to file: {time.monotonic() - time1}')

Time to dump all_dfs to file: 1.115097158588469


In [16]:
all_dfs_loaded = pickle.load(open('pickles/all_dfs.pkl', 'rb'))
all_dfs_loaded

[             ds   GISJOIN   y
 0    2020-03-27  G0100010   0
 1    2020-03-25  G0100010   3
 2    2020-03-28  G0100010   0
 3    2020-03-29  G0100010   0
 4    2020-03-30  G0100010   1
 ..          ...       ...  ..
 344  2021-03-06  G0100010  11
 345  2021-03-04  G0100010  11
 346  2021-03-07  G0100010   3
 377  2021-01-16  G0100010  51
 443  2021-01-26  G0100010  33
 
 [349 rows x 3 columns],
               ds   GISJOIN   y
 347   2020-03-15  G0100030   0
 348   2020-03-14  G0100030   1
 349   2020-03-16  G0100030   0
 350   2020-03-18  G0100030   0
 351   2020-03-17  G0100030   0
 ...          ...       ...  ..
 1203  2021-03-05  G0100030  17
 1204  2021-03-07  G0100030  20
 1205  2021-03-06  G0100030  25
 1206  2021-03-03  G0100030  66
 1208  2021-03-04  G0100030  17
 
 [359 rows x 3 columns],
               ds   GISJOIN   y
 500   2021-01-22  G0100050  15
 501   2021-01-23  G0100050  12
 502   2021-01-24  G0100050  22
 503   2021-01-20  G0100050  22
 504   2021-01-13  G0100050   

In [20]:
all_dfs[0]

,ds,GISJOIN,y
0,2020-03-27,G0100010,0
1,2020-03-25,G0100010,3
2,2020-03-28,G0100010,0
3,2020-03-29,G0100010,0
4,2020-03-30,G0100010,1
...,...,...,...
344,2021-03-06,G0100010,11
345,2021-03-04,G0100010,11
346,2021-03-07,G0100010,3
377,2021-01-16,G0100010,51


In [25]:
# model = Prophet(
#     interval_width=0.95,
#     growth='linear',
#     daily_seasonality=False,
#     weekly_seasonality=True,
#     yearly_seasonality=True,
#     seasonality_mode='multiplicative'
# )

# model.fit(df0)

def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True
    )
    m.fit(df_train)
    df_train_future = m.make_future_dataframe(periods=365)
    df_train_forecast = m.predict(df0_future)

    return df_train_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

In [27]:
all_results = []

count = 0
time1 = time.monotonic()
for df in all_dfs:
    count += 1
    if count % 5 == 0:
        print(count, end=', ')
    result = predict(df)
    all_results.append(result)
    
print(f'Time to train all dfs: {time.monotonic() - time1}')
print(f'#Result Dataframes: {len(all_results)}')

5, 10, 15, 20, 25, 30, 35, 40, 45, 

ValueError: Dataframe has less than 2 non-NaN rows.

In [26]:
# m.plot_components(df0_forecast)

In [25]:
# import plotly.graph_objs as go
# plot_plotly(m, df0_forecast)

In [31]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *


result_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True)
])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def forecast_store_item(history_pd):

    # instantiate the model, configure the parameters
    model = Prophet()

    # fit the model
    model.fit(history_pd)

    # configure predictions
    future_pd = model.make_future_dataframe(
        periods=90,
        freq='d',
        include_history=True
    )

    # make predictions
    results_pd = model.predict(future_pd)

    # . . .

    # return predictions
    return results_pd

In [32]:
result_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True)
])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def temp(history_pd):

    # instantiate the model, configure the parameters
    m = Prophet()
    m.fit(df0)
    df0_future = m.make_future_dataframe(periods=365)
    df0_forecast = m.predict(df0_future)

    return df0_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [44]:
# df0 = df0[["ds", "y"]]
# df0.set_index('ds')
# temp(df0[["ds", "GISJOIN", "y", "deaths"]])
# df0

# results = (
#     df.groupBy('GISJOIN')
#     .apply(temp)
# )
# df0_ = df0.withColumn('GISJOIN', lit(None))

# from pyspark.sql import SparkSession

# spark = SparkSession \
#     .builder\
#     .master('local')\
#     .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/sustaindb.covid_county_formatted')\
#     .getOrCreate()

# spark_df = spark.read.format("mongo").option("uri",
#     "mongodb://localhost/sustaindb.covid_county_formatted").load()
# spark_df
#     .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1.jar')\

# spark.sparkContext.addPyFile('./jars/mongo-spark-connector_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-core_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-sql_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-mllib_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/bson-4.0.5.jar')
# spark.sparkContext.addPyFile('./jars/mongo-java-driver-3.12.5.jar')

# df1 = spark.read.format('com.mongodb.spark.sql.DefaultSource')\
#             .option('database', 'sustaindb')\
#             .option('collection', 'covid_county_formatted')\
#             .load()

df = spark.read.format('json').load('./covid_county.json')

In [45]:
df = df.select('GISJOIN', 'cases', 'deaths', 'date', 'formatted_date')
df.show()

+--------+-----+------+----------+--------------------+
| GISJOIN|cases|deaths|      date|      formatted_date|
+--------+-----+------+----------+--------------------+
|G0100010|    3|     0|2020-03-25|[2020-03-25T00:00...|
|G0100010|    0|     0|2020-03-28|[2020-03-28T00:00...|
|G0100010|    0|     0|2020-03-29|[2020-03-29T00:00...|
|G0100010|    1|     0|2020-03-30|[2020-03-30T00:00...|
|G0100010|    0|     0|2020-03-31|[2020-03-31T00:00...|
|G0100010|    3|     0|2020-04-01|[2020-04-01T00:00...|
|G0100010|    0|     0|2020-04-04|[2020-04-04T00:00...|
|G0100010|    0|     0|2020-04-05|[2020-04-05T00:00...|
|G0100010|    0|     1|2020-04-06|[2020-04-06T00:00...|
|G0100010|    0|     0|2020-04-02|[2020-04-02T00:00...|
|G0100010|    0|     0|2020-04-07|[2020-04-07T00:00...|
|G0100010|    0|     0|2020-04-08|[2020-04-08T00:00...|
|G0100010|    5|     0|2020-04-09|[2020-04-09T00:00...|
|G0100010|    0|     0|2020-04-10|[2020-04-10T00:00...|
|G0100010|    2|     0|2020-04-11|[2020-04-11T00

In [46]:
# df0 = df0.rename(columns = {
#     'date': 'ds',
#     'cases': 'y'
# })
df_cases = df.select('GISJOIN', 'date', 'cases').withColumnRenamed('date', 'ds').withColumnRenamed('cases', 'y')
# df_cases.show()

results = (
    df_cases.groupBy('GISJOIN')
    .apply(temp)
)
results.toPandas()

Py4JJavaError: An error occurred while calling o530.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 44, 10.253.202.195, executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at io.netty.util.internal.PlatformDependent.directBuffer(PlatformDependent.java:490)
	at io.netty.buffer.NettyArrowBuf.getDirectBuffer(NettyArrowBuf.java:243)
	at io.netty.buffer.NettyArrowBuf.nioBuffer(NettyArrowBuf.java:233)
	at io.netty.buffer.ArrowBuf.nioBuffer(ArrowBuf.java:245)
	at org.apache.arrow.vector.ipc.message.ArrowRecordBatch.computeBodyLength(ArrowRecordBatch.java:222)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.serialize(MessageSerializer.java:240)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeRecordBatch(ArrowWriter.java:132)
	at org.apache.arrow.vector.ipc.ArrowWriter.writ

In [3]:
 from pyspark.sql import SparkSession 

 spark = SparkSession.builder.master("spark://lattice-100:8079").getOrCreate() 
 print(spark.sparkContext.version)

3.0.1
